# Guided exercise 8: Input and Output

With monadic programming we are able to handle IO (and more things we are used to in imperative programming)

The basic IO actions are `getChar :: IO Char`, `putChar :: Char -> IO ()` and `return :: a -> IO a`

We call them actions because they have side effects (reading something from the keyboard or printing something to the screen) in addition to returning something as any other Haskell function. The returning value is always `IO a`

In [2]:
-- Checking the types
:t getChar
:t putChar
:t return 

getChar :: IO Char

putChar :: Char -> IO ()

return :: forall (m :: * -> *) a. Monad m => a -> m a

In [3]:
-- Some examples
getChar
putChar 'a'
-- Be carefull this is not an Int, but a Monad Int
return 4
:t return 4
-- This will raise an error
-- 3 + (return 4)

'c'

a

4

return 4 :: forall {m :: * -> *} {a}. (Monad m, Num a) => m a

 **Exercise 1.** Create a function `multiplyPut Int` that receives a number, multiplies it by 2 and puts it on IO using `putChar`. Guess the type.

In [4]:
-- First version, only the first character is printed
multiplyPut :: (Show a, Num a) => a -> IO ()
multiplyPut a = putChar $ head $ show $ a * 2

In [5]:
multiplyPut 12
multiplyPut 1.3

2

2

In [27]:
-- To be able to show all characters we need to use a do
putChars :: String -> IO ()
putChars [] = putChar '\n'
putChars (x:xs) = do
    putChar x
    putChars xs
    
multiplyPut' :: (Show a, Num a) => a -> IO ()
multiplyPut' a = putChars $ show $ a * 2

In [8]:
multiplyPut' 12
multiplyPut' 1.3

24

2.6

In [28]:
:t putChars

putChars :: String -> IO ()

**Performing several I/O operations in a row.** If we want to perform several I/O operations one after the other, we need to use the `do`. This will make our function to look like an imperative program.

In [7]:
-- Function that writes all the characters of a string with - between them
-- The type is IO () as the base case is IO ()
writeChars :: String -> IO ()
writeChars [x] = putChar x
writeChars (x:xs) = do
        -- Indentation must be the same for all the do block
        putChar x
        putChar '-'
        writeChars xs

In [8]:
writeChars "hello"

h-e-l-l-o

**Exercise 2.** Create a `printVowels String` function that prints the vowels of a word separated by "-"

In [5]:
vowel :: Char -> Bool
vowel x = x `elem` "aeiouAEIOU"

filterVowel :: String -> String
filterVowel = filter vowel

printVowels :: String -> IO ()
printVowels [x] = if vowel x then putChar x else return ()
printVowels (x:xs) = if not (vowel x) then printVowels xs
        else do
            putChar x
            putChar '_'
            printVowels xs

printVowels' xs = printVowels $ filterVowel xs

Line 8: Use when
Found:
if vowel x then putChar x else return ()
Why not:
Control.Monad.when (vowel x) $ putChar x

In [7]:
filterVowel "hello how are you, I am OK" 
printVowels "hello how are you, I am OK"
printVowels "hello how are you, I am OK like you"
printVowels' "hello how are you, I am OK"
printVowels' "hello how are you, I am OK like you"

"eooaeouIaO"

e_o_o_a_e_o_u_I_a_O_

e_o_o_a_e_o_u_I_a_O_i_e_o_u

e_o_o_a_e_o_u_I_a_O

e_o_o_a_e_o_u_I_a_O_i_e_o_u

The `getChar` function does not return a `Char` but an `IO Char` which is not compatible. How do we work with the `Char` in `getChar :: IO Char`? 

In [1]:
-- Function that attempts to read a char and return it
-- Notice it returns a monad of IO Char but not the Char (we'll work with monads next week)
wrongReadChar :: Monad m => m (IO Char)
wrongReadChar = return getChar

We need to use the `<-` operator inside a `do`. Notice the `=` does not work for `IO` as you would get the whole `IO Char`

In [14]:
-- Function that reads two chars and returns a String with them
-- Notice String is defined as type String = [Char], so we could also put IO [Char]
readTwoChars :: IO String 
readTwoChars = do 
    -- Indentation must be equal for all the do block
    -- The <- operator binds the result of an IO to a name
    x <- getChar
    putStrLn $ "The first character is " ++ [x]
    -- This is to discard the /n
    getChar
    y <- getChar
    putStrLn $ "The second character is " ++ [y]
    -- This is to discard the /n
    getChar
    return [x, y]

In [15]:
readTwoChars

The first character is a
The second character is b
"ab"

**Exercise 3.** Create a function `equalChars` that reads two chars and returns if they are equal or not

In [3]:
equalChars :: IO Bool
equalChars = do         
                -- We'll use the previous one
                x <- readTwoChars
                putStrLn $ "The first character is " ++ show (head x)
                putStrLn $ "The second character is " ++ show (last x)
                return (head x == last x)

In [4]:
equalChars

The first character is 'a'
The second character is 'a'
True

The `getChar` function is generalized by `getLine` (see slides). We have also `putStr` and `putStrLn`

In [8]:
-- Try it without the do, to see what happens
putStrLn "Enter you name"
do 
    x <- getLine
    putStr "hello " 
    putStr x
    putStrLn "! How are you?"

-- Types
:t getLine
:t putStr

Enter you name

hello Pepe! How are you?

getLine :: IO String

putStr :: String -> IO ()

With `getChar` and `getLine` we read a `Char` and a `String` respectively, but how do we read numbers? We can read a `String` and use `let` and the `read` method:

In [10]:
evenInput :: IO String
evenInput = do
            putStrLn "Enter a number"
            sn <- getLine
            putStrLn ("You entered " ++ sn)
            if even (read sn :: Int) then return "Even" else return "Odd"

In [11]:
evenInput

Enter a number
You entered 45
"Odd"

**Exercise 4.** Create a function that asks for two integer numbers, adds them and returns the result.

In [12]:
addNumbers :: IO Int
addNumbers = do
        putStrLn "Enter the first number"
        sn1 <- getLine
        putStrLn sn1
        putStrLn "Enter the second number"
        sn2 <- getLine
        putStrLn sn2
        -- We can use let to make things easier to read
        let n1 = read sn1 :: Int
            n2 = read sn2 :: Int
        putStrLn (sn1 ++ " + " ++ sn2 ++ " = " ++ show (n1 + n2))
        return (n1 + n2)

In [19]:
addNumbers

Enter the first number
5
Enter the second number
22
5 + 22 = 27
27

How do we get the read numbers out of the function? In other languages we are used to do something like 
 `result = readNumber()` and then work with `result`. In Haskell we cannot do it, but we can invoke another function on a read value:

In [13]:
inbounds :: Int -> Int -> Int -> Bool
inbounds lb up val = lb <= val && up >= val

checkBound :: IO Bool
checkBound =
    do
    putStrLn "Enter the lower bound"
    lb <- getLine
    putStrLn $ "You entered " ++  lb
    putStrLn "Enter upper bound"
    ub <- getLine
    putStrLn $ "You entered " ++  ub
    putStrLn "Enter the value"
    val <- getLine
    putStrLn $ "You entered " ++  val
    return $ inbounds (read lb::Int) (read ub :: Int) (read val :: Int)

In [14]:
checkBound

Enter the lower bound
You entered 2
Enter upper bound
You entered 10
Enter the value
You entered 4
True

Check the result of the following program. As you see in Haskell `return` is not like in imperative languages, it should be read as *wrap the datum to an IO type*

In [19]:
function = do
    putStrLn "hello"
    return 4
    putStrLn "bye"

Line 3: Redundant return
Found:
return 4
Why not:

In [20]:
function

hello
bye

In [24]:
-- print is an alias for putStrLn . show
putStrLn . show $ 3
print 3

Line 2: Use print
Found:
putStrLn . show
Why not:
print

3

3